<a href="https://colab.research.google.com/github/danidads/Bootcamp_IA_Alura/blob/main/ChatBot_Consultora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
%pip -q install google-genai

In [36]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [38]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [39]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [40]:
# Instala Pillow, se ainda não tiver
!pip install pillow
from PIL import Image, ImageDraw, ImageFont
import os


In [41]:
from PIL import Image, ImageDraw, ImageFont
import os


In [42]:
import re
import openai
import textwrap
from PIL import Image
from io import BytesIO

In [54]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
from datetime import date
from IPython.display import display
from tabulate import tabulate
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import warnings

warnings.filterwarnings("ignore")

In [44]:

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap

In [45]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [46]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [47]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search) e site oficial (https://www.marykay.com.br/)
        para recuperar as útimas notícias de lançamentos Mary Kay, cuidados com a pele e maquiagem muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Traga informações de cuidados com a pele e maquiagem que possa fazer uma junção com os produtos Mary Kay.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [48]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais na área de beleza Mary Kay. Com base na lista de lançamentos mais recentes e relevantes buscados, você deve:
        usar a ferramenta de busca do Google (google_search) e site oficial (https://www.marykay.com.br/) para criar um plano sobre quais são os pontos mais relevantes que poderíamos abordar em um post sobre cada um deles.
        Você também pode usar o (google_search) e site oficial (https://www.marykay.com.br/) para encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [49]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a Elizabete Dias, a melhor Consultora Independente Mary Kay de Goiânia, Brasil.
            Por ter um público de jovem senhoras, entre 30 e 80 anos, use imagens adequadas.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva 5 rascunhos de post para Instagram sobre o tema indicado.
            Quero que traga referências do site oficial (https://www.marykay.com.br/).
            Os posts devem ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [50]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais para Consultoras Independente Mary Kay, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [51]:
# ######################################################
# # --- Agente 5: Criador de Página Web com Canva --- #
# ######################################################
# def agente_web_canva(tema_web):
#     web_designer = Agent(
#         name="agente_web_canva",
#         model="gemini-2.0-flash",
#         description="Agente especializado na criação de designs de páginas web usando a ferramenta Canva.",
#         tools=[canva, google_search],
#         instruction="""
#         Você é um web designer criativo e especialista em usar a ferramenta Canva para criar designs de páginas web atraentes e informativas para consultoras de beleza Mary Kay.

#         Com base no tema fornecido pelo usuário, sua tarefa é utilizar a ferramenta 'canva' para gerar um design inicial de página web.

#         Você pode usar a ferramenta 'google_search' para buscar inspiração de designs de páginas web de beleza ou exemplos de páginas de consultoras Mary Kay, se necessário.

#         O design deve ser visualmente agradável, fácil de navegar e adequado para apresentar informações sobre produtos Mary Kay, dicas de beleza ou informações de contato da consultora.

#         Ao final, forneça uma descrição do design da página web que você criou e, se possível, inclua um link ou informações sobre como acessar ou editar o design no Canva.
#         """
#     )

#     entrada_do_agente_web = f"Tema para a página web: {tema_web}"
#     design_web = call_agent(web_designer, entrada_do_agente_web)
#     return design_web



In [55]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 5 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("⚠️ Por favor, digite um tópico válido!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 5 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: olhos
Maravilha! Vamos então criar o post sobre novidades em olhos

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para fornecer as informações mais recentes sobre lançamentos de produtos Mary Kay para os olhos e dicas de cuidados com a pele e maquiagem, vou realizar algumas pesquisas no Google.
> 
> 
> Com base nas minhas buscas, aqui estão alguns lançamentos e tendências da Mary Kay para os olhos que podem ser relevantes para você:
> 
> **Lançamentos e Produtos Mary Kay para Olhos:**
> 
> 1.  **Lápis Retrátil para os Olhos à Prova d'Água Mary Kay®:** Este lápis foi lançado em 2024 e está disponível em três cores clássicas: preto, marrom e azul. Ele oferece alta pigmentação e uma textura cremosa, ideal para delineados finos e esfumados. A fórmula à prova d'água garante resistência a transferências e borrões, proporcionando um olhar marcante durante todo o dia.
> 2.  **Máscara de Pestanas Great Heights™:** NOVA! Máscara de Pestanas Great Heights™.
> 3.  **Máscara de Cílios:** Uma máscara de cílios está incluída em um dos kits de início para novas consultoras.
> 4.  **Desmaquilhante de olhos Mary Kay®:** Para uma remoção suave da maquiagem dos olhos.
> 5.  **Gel Suavizante para a Área dos Olhos Mary Kay:** Contém extratos botânicos tonificantes, firmadores e redutores da aparência de inchaço. Aumenta a hidratação da pele em 130%, acalmando e tonificando a área delicada ao redor dos olhos.
> 
> **Tendências de Maquiagem para Olhos em 2025:**
> 
> *   **Olhos Marcantes:** Os olhos continuam sendo o foco principal na maquiagem. Delineadores gráficos, sombras metalizadas e cílios postiços volumosos estão em alta.
> *   **Cores Vibrantes:** Delineadores coloridos em tons de verde, azul, laranja e rosa são uma forte tendência, usados em traços gráficos, gatinho ou apenas um toque no canto dos olhos. Sombras também seguem a mesma vibe, com aplicações descomplicadas e combinações improváveis.
> *   **Sombras Metalizadas:** As sombras metalizadas destacam os olhos, criando looks sofisticados e elegantes para qualquer ocasião.
> *   **Maquiagem Minimalista:** Uma abordagem mais natural e sustentável, com foco em produtos que realçam a beleza natural. Menos camadas de maquiagem e mais foco em texturas leves e acabamentos luminosos.
> *   **Delineado de Gatinho:** O clássico delineado de gatinho continua em alta, usado com traços coloridos, formatos variados e até mesmo sombreados. Delineados gráficos duplos ou combinados com sombras coloridas também são opções.
> 
> **Cuidados com a Pele ao Redor dos Olhos:**
> 
> *   **Hidratação:** A pele ao redor dos olhos é fina e sensível, sendo a primeira a mostrar sinais de cansaço e envelhecimento. A hidratação é fundamental para prevenir linhas finas e rugas.
> *   **Ativos Poderosos:** Ao escolher produtos para a área dos olhos, procure por ingredientes como Ácido Hialurônico (hidratação), Cafeína (redução de olheiras e inchaço) e Vitamina E (antioxidante).
> *   **Cremes para a Área dos Olhos:** Produtos específicos, como séruns e máscaras, podem oferecer benefícios adicionais. Ao escolher um produto, considere o tipo de pele e as preocupações individuais.
> *   **Aplicação Correta:** Aplique os produtos suavemente, usando o dedo anelar para evitar pressão excessiva. Evite a área próxima ao canal lacrimal para prevenir irritações.
> *   **Rotina Diária:** Incorpore os cuidados com a área dos olhos tanto na rotina matinal quanto na noturna para garantir proteção e hidratação contínuas.
> 
> **Produtos Mary Kay para Cuidados com a Área dos Olhos:**
> 
> *   **Gel Suavizante para a Área dos Olhos Mary Kay:** Revitaliza e refresca a aparência de cansaço ao redor dos olhos, aumentando a hidratação da pele.
> *   **Creme para Área dos Olhos Anti-sinais Timewise 3D Mary Kay:** Combate múltiplos sinais visíveis de envelhecimento e fadiga da pele, melhorando linhas finas, opacidade, firmeza e textura da pele.
> *   **Creme para a Área dos Olhos Volu-Firm Mary Kay TimeWise Repair:** Para pele sensível de 14g 40+ anos.
> 
> Lembre-se que a Mary Kay oferece uma variedade de produtos para os olhos, desde sombras e delineadores até produtos de cuidados com a pele específicos para a área dos olhos. As tendências de maquiagem para 2025 apontam para olhos marcantes e coloridos, combinados com uma pele natural e bem cuidada.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Com base nas informações sobre os lançamentos e tendências da Mary Kay para os olhos, podemos criar um plano de conteúdo diversificado e relevante para as redes sociais. Aqui estão alguns pontos que poderíamos abordar em posts sobre cada um dos temas:
> 
> **1. Lápis Retrátil para os Olhos à Prova d'Água Mary Kay®:**
> 
> *   **Destaque:** Lançamento recente com alta pigmentação e fórmula à prova d'água.
> *   **Pontos a abordar:**
>     *   Apresentar as três cores disponíveis (preto, marrom e azul) com fotos e vídeos demonstrando a intensidade da cor.
>     *   Mostrar a textura cremosa do lápis e como ele desliza suavemente na pele, facilitando a aplicação.
>     *   Criar tutoriais de maquiagem com delineados finos e esfumados, mostrando a versatilidade do produto.
>     *   Realizar testes de resistência à água para comprovar a durabilidade e a prova d'água do lápis.
>     *   Comparar com outros lápis de olho no mercado, destacando os benefícios e diferenciais do produto Mary Kay.
> 
> **2. Máscara de Pestanas Great Heights™:**
> 
> *   **Destaque:** Lançamento da nova máscara de pestanas.
> *   **Pontos a abordar:**
>     *   Focar no diferencial da máscara, como o aumento do volume e alongamento das pestanas.
>     *   Criar tutoriais mostrando o antes e depois da aplicação, destacando o impacto no olhar.
>     *   Apresentar a fórmula da máscara, mencionando se é à prova d'água, de longa duração e se possui ingredientes que cuidam das pestanas.
>     *   Fazer um vídeo comparando a máscara com outras opções no mercado, mostrando os resultados em diferentes tipos de pestanas.
> 
> **3. Máscara de Cílios (Kit de Início):**
> 
> *   **Destaque:** Produto incluído no kit de início para novas consultoras.
> *   **Pontos a abordar:**
>     *   Mostrar os benefícios da máscara de cílios, como alongamento, volume e definição.
>     *   Criar um vídeo mostrando como a máscara pode ser usada para criar diferentes looks, desde o natural até o mais dramático.
>     *   Explicar como a máscara de cílios pode ser uma ferramenta importante para as novas consultoras demonstrarem os produtos Mary Kay.
> 
> **4. Desmaquilhante de Olhos Mary Kay®:**
> 
> *   **Destaque:** Remoção suave da maquiagem dos olhos.
> *   **Pontos a abordar:**
>     *   Demonstrar a eficácia do desmaquilhante na remoção de diferentes tipos de maquiagem, incluindo produtos à prova d'água.
>     *   Mostrar como o produto é suave e não irrita a área dos olhos, ideal para peles sensíveis.
>     *   Explicar como usar corretamente o desmaquilhante para evitar irritações e garantir uma limpeza completa.
>     *   Destacar os ingredientes que ajudam a hidratar e proteger a pele ao redor dos olhos.
> 
> **5. Gel Suavizante para a Área dos Olhos Mary Kay:**
> 
> *   **Destaque:** Contém extratos botânicos e aumenta a hidratação da pele.
> *   **Pontos a abordar:**
>     *   Explicar como o gel suavizante ajuda a reduzir o inchaço e a aparência de cansaço ao redor dos olhos.
>     *   Mostrar como o produto pode ser usado na rotina diária de cuidados com a pele, tanto pela manhã quanto à noite.
>     *   Destacar os extratos botânicos presentes na fórmula e seus benefícios para a pele.
>     *   Realizar testes de hidratação para comprovar o aumento da hidratação da pele em 130%.
> 
> **6. Tendências de Maquiagem para Olhos em 2025:**
> 
> *   **Destaque:** Olhos marcantes, cores vibrantes e sombras metalizadas.
> *   **Pontos a abordar:**
>     *   Criar tutoriais de maquiagem inspirados nas tendências de 2025, mostrando como usar delineadores coloridos, sombras metalizadas e cílios postiços volumosos.
>     *   Apresentar produtos Mary Kay que se encaixam nessas tendências, como delineadores em gel coloridos e sombras cintilantes.
>     *   Inspirar o público a experimentar novas cores e estilos, mostrando que a maquiagem pode ser divertida e criativa.
> 
> **7. Cuidados com a Pele ao Redor dos Olhos:**
> 
> *   **Destaque:** Hidratação, ativos poderosos e rotina diária.
> *   **Pontos a abordar:**
>     *   Explicar a importância da hidratação para prevenir linhas finas e rugas na área dos olhos.
>     *   Apresentar ingredientes como Ácido Hialurônico, Cafeína e Vitamina E, destacando seus benefícios para a pele.
>     *   Criar um guia completo de cuidados com a pele ao redor dos olhos, incluindo dicas de aplicação correta dos produtos e rotina diária.
> 
> **Escolha do tema mais relevante:**
> 
> Com base nas pesquisas, o tema mais relevante a ser abordado no post é o **Lápis Retrátil para os Olhos à Prova d'Água Mary Kay®**. Por ser um lançamento recente e possuir características como alta pigmentação, textura cremosa e fórmula à prova d'água, ele se destaca como um produto inovador e versátil. Além disso, a possibilidade de criar tutoriais de maquiagem com diferentes estilos de delineado torna o tema ainda mais interessante para o público.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> Perfeito! Com base no seu plano e priorizando o **Lápis Retrátil para os Olhos à Prova d'Água Mary Kay®**, aqui estão 5 rascunhos de posts para o Instagram da Elizabete Dias:
> 
> **Rascunho 1: (Carrossel com fotos e vídeos)**
> 
> *   **Imagem 1:** Close-up do lápis nas 3 cores (preto, marrom, azul) com fundo clean.
> *   **Imagem 2:** Vídeo curto demonstrando a aplicação suave do lápis na pálpebra.
> *   **Imagem 3:** Foto de um delineado gatinho impecável feito com o lápis preto.
> *   **Imagem 4:** Teste de resistência à água (borrifando água no delineado e mostrando que não borra).
> *   **Legenda:** ✨ LANÇAMENTO BABADEIRO! ✨ Chegou o Lápis Retrátil à Prova d'Água Mary Kay! 😱 Cores intensas, textura super cremosa que desliza fácil e... PODE CHORAR À VONTADE! 😉 Disponível em preto, marrom e azul pra você arrasar em qualquer look. Qual sua cor favorita? 👇 #MaryKayBrasil #LapisRetratilMK #MakeAProvaDAgua #LancamentoMaryKay
> 
> **Rascunho 2: (Vídeo Reel curto)**
> 
> *   **Vídeo:** Transição rápida mostrando a Elizabete com o rosto limpo e, em seguida, com um delineado poderoso feito com o lápis.
> *   **Áudio:** Música trendy e animada.
> *   **Texto na tela:** "Delineado perfeito em segundos? ✅ À prova d'água? ✅ Cores incríveis? ✅  Lápis Retrátil Mary Kay!"
> *   **Legenda:**  Acorda, gata! 🤩 Seu delineado dos sonhos agora é REALIDADE com o novo Lápis Retrátil Mary Kay! ✨ Prático, pigmentado e à prova de tudo! 😉 Quer experimentar? Me chama no direct! 💖 #MaryKayGoiania #DelineadoPerfeito #MakeUpTutorial #ElizabeteDiasMK
> 
> **Rascunho 3: (Imagem única com frase impactante)**
> 
> *   **Imagem:**  Olho com um delineado gráfico moderno usando o lápis azul.
> *   **Legenda:**  💙 AZUL POWER! 💙  Quem disse que make discreta é a única opção? 😉 Arrase com o Lápis Retrátil Azul Mary Kay e mostre sua personalidade! ✨  Alta pigmentação, fácil de aplicar e à prova d'água!  😱  Peça já o seu! 💋 #MaryKayBrasil #OlhosDestaque #AzulVibrante #ConsultoraMaryKay
> 
> **Rascunho 4: (Carrossel comparativo)**
> 
> *   **Imagem 1:** Foto do Lápis Retrátil Mary Kay ao lado de um lápis de outra marca.
> *   **Imagem 2:**  Vídeo mostrando a aplicação dos dois lápis em um papel (demonstrando a pigmentação superior do Mary Kay).
> *   **Imagem 3:** Teste de borrão: passando o dedo sobre os dois traços (mostrando que o Mary Kay não borra).
> *   **Legenda:**  🤔 LÁPIS DE OLHO: QUAL A MELHOR OPÇÃO? 🤔  Não se engane! 😉  O Lápis Retrátil Mary Kay oferece: ✅ Pigmentação intensa ✅ Textura macia ✅ Longa duração ✅ À prova d'água!  ✨  Invista na sua beleza!  💖  Fale comigo e garanta o seu! #MaryKayGoiania #ComparativoMake #LapisDeOlho #QualidadeMaryKay
> 
> **Rascunho 5: (Story + Post no Feed)**
> 
> *   **Story:** Enquete: "Qual sua cor favorita do novo Lápis Retrátil Mary Kay? Preto, Marrom ou Azul?".  Use um GIF chamativo e incentive a participação.
> *   **Post no Feed (após 24h):**
>     *   **Imagem:** Montagem com as três cores do lápis e o resultado da enquete no Story.
>     *   **Legenda:**  🔥 A COR MAIS AMADA FOI... 🔥  [Resultado da enquete]!  😍  Mas a verdade é que TODAS as cores do Lápis Retrátil Mary Kay são incríveis!  ✨  E você, já escolheu a sua? 😉  Corre pro direct e aproveite!  💋 #MaryKayBrasil #EnqueteMake #LapisColorido #ConsultoraDeBeleza
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!


--------------------------------------------------------------


In [ ]:
from google.colab import drive
drive.mount('/content/drive')